In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.dummy import DummyRegressor
import xgboost as xgb

In [3]:
data_orig = pd.read_csv('../MA_PREDICTOR/data/ma_data_car_clean.csv')

In [4]:
data = data_orig[(data_orig.car > -0.3) & (data_orig.car < 0.3)]

In [5]:
data

,consideration_offered,shares_acquired,shares_at_announcement,acquiror_code,target_code,target_status,month,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target,car
0,Cash,full,no,50102030,50103030,others,1,cross_border,business_sector,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,0.006854
1,Other,full,no,54201030,63103010,others,1,cross_border,not_related,Consumer Non-Cyclicals,Personal & Household Products & Services,Academic & Educational Services,Academic & Educational Services,-0.010266
2,Other,full,no,57201030,57201020,others,1,cross_border,industry_group,Technology,Software & IT Services,Technology,Software & IT Services,0.007746
3,Cash,full,no,52102010,51101010,others,1,national,not_related,Industrials,Industrial Goods,Basic Materials,Chemicals,-0.011133
4,Cash,not_full,no,50102030,50102030,public,1,cross_border,industry,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,-0.003971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18219,Cash,full,no,52102010,59103010,others,8,cross_border,not_related,Industrials,Industrial Goods,Utilities,Utilities,0.020108
18220,Other,full,no,53203020,53205020,others,8,cross_border,business_sector,Consumer Cyclicals,Cyclical Consumer Products,Consumer Cyclicals,Cyclical Consumer Products,-0.040156
18221,Other,full,no,54301020,57201010,others,8,national,not_related,Consumer Non-Cyclicals,Food & Drug Retailing,Technology,Software & IT Services,-0.003545
18222,Other,full,no,55101010,52203030,others,8,cross_border,not_related,Financials,Banking & Investment Services,Industrials,Industrial & Commercial Services,-0.025992


In [6]:
data.columns

Index(['consideration_offered', 'shares_acquired', 'shares_at_announcement',
       'acquiror_code', 'target_code', 'target_status', 'month',
       'cross_border', 'relatedness', 'economic_sector_ac',
       'business_sector_ac', 'economic_sector_target',
       'business_sector_target', 'car'],
      dtype='object')

In [7]:
y=data['car']

In [8]:
X=data[['consideration_offered', 'shares_acquired', 'shares_at_announcement', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target', 'target_status', 
       'business_sector_target']]

# Splitting of the dataset

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

# Pipeline and GridSearch

In [10]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [11]:
cat_features=['consideration_offered', 'shares_acquired', 'shares_at_announcement', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target', 'target_status', 
       'business_sector_target']

In [12]:
preprocessor = ColumnTransformer([
    ('cat_transformer', cat_transformer, cat_features)])

## Baseline

In [53]:
baseline_model_mean = DummyRegressor(strategy="mean") 

In [54]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='r2').mean()

-0.0005477136768966328

In [55]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-0.0021237193472723316

In [90]:
baseline_model_mean.score(X_test, y_test)

NotFittedError: This DummyRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## ElasticNet

In [46]:
# Hyperparameter Grid
grid = {'model__alpha': [0.1, 0.2, 0.3, 0.4],
        'model__l1_ratio': [0,0.05, 0.1, 0.15, 0.2],
        'model__max_iter': [10000]}

In [47]:
# Combine preprocessor and linear model in pipeline
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', ElasticNet())])

In [48]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [49]:
# Fit data to Grid Search
search.fit(X_train,y_train);

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13.509962771944553, tolerance: 0.00270848662194968
  model = cd_fast.sparse_enet_coordinate_descent(


In [50]:
# Best score 1
search.best_score_

-0.002120288949990078

## RandomForest

In [55]:
RandomForestRegressor().get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.823037600169421, tolerance: 0.002171952132502156
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.909977424148362, tolerance: 0.0021893696612592245
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.910393640038457, tolerance: 0.002189369661

In [26]:
# Hyperparameter Grid
grid={'model__bootstrap': [True, False],
      'model__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
      'model__max_features': ['auto', 'sqrt'],
      'model__min_samples_leaf': [1, 2, 4, 10],
      'model__min_samples_split': [2, 5, 10],
      'model__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [27]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor())])

In [28]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [29]:
# Fit data to Grid Search
search.fit(X_train,y_train);

KeyboardInterrupt: 

In [61]:
# Best score 1
search.best_score_

-0.0025065721534876877

In [ ]:
# Best score 2
search.best_score_

In [ ]:
# Best score 3
search.best_score_

In [62]:
# Best Params
search.best_params_

{'model__n_estimators': 150}

## XGBoost

In [13]:
model= XGBRegressor()

In [20]:
# Hyperparameter Grid
grid = {'model__n_estimators': [500, 800, 1000],
        'model__learning_rate':  [0.1, 0.2 , 0.3, 0.4],
        'model__max_depth': [1, 2, 3]}

In [21]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [58]:
cross_val_score(pipe, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-0.002258528474975258

In [22]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [23]:
# Fit data to Grid Search
search.fit(X_train,y_train);

In [71]:
# Best score 1
search.best_score_

-0.002129517642842963

In [72]:
search.best_params_

{'model__learning_rate': 0.7, 'model__max_depth': 3, 'model__n_estimators': 5}

In [79]:
# Best score 2
search.best_score_

-0.002122335369717998

In [80]:
search.best_params_

{'model__learning_rate': 0.7, 'model__max_depth': 1, 'model__n_estimators': 6}

In [85]:
# Best score 3
search.best_score_

-0.0021306478955788618

In [86]:
search.best_params_

{'model__learning_rate': 0.5,
 'model__max_depth': 1,
 'model__n_estimators': 1000}

In [18]:
# Best score 4
search.best_score_

-0.0021304504982820943

In [19]:
search.best_params_

{'model__learning_rate': 0.3,
 'model__max_depth': 1,
 'model__n_estimators': 1000}

In [24]:
# Best score 5
search.best_score_

-0.002124521752699194

In [25]:
search.best_params_

{'model__learning_rate': 0.1,
 'model__max_depth': 1,
 'model__n_estimators': 500}